In [2]:
import os,sys

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [24]:
out_dir = '/mnt/store1/oliverphilcox/3PCF_QPM_v3/'
qpm_proc_root = '/mnt/store1/oliverphilcox/QPM_proc/'
i=0
for j in range(1,301):
    gal_path = qpm_proc_root+'qpm_galaxy_'+str(j).zfill(4)+'.xyzw'
    if os.path.exists(gal_path):
        os.system('cp %s %s'%(gal_path,out_dir+'mock_'+str(i)))
        i+=1

## Compute mean weights

In [105]:
all_N_gal=[]
all_weights=[]
for i in range(290):
    infile = out_dir+'mock_%d'%i
    with open(infile) as inn:
        for l,line in enumerate(inn):
            pass
    this_N_gal=l+1
    all_N_gal.append(this_N_gal)
    this_weights=np.zeros(l+1,dtype=float)
    with open(infile) as inn:
        for l,line in enumerate(inn):
            w=float(line.split()[-1])
            this_weights[l] = w
    all_weights.append(this_weights)

In [109]:
np.save('/mnt/store1/oliverphilcox/3PCF_QPM_v3/all_N_gal.npy',all_N_gal)

In [110]:
sum_w = [w.sum() for w in all_weights]
mean_N_gal = np.mean(all_N_gal)
mean_sum_w = np.mean(sum_w)
print("Sum-weighted: %.2f"%mean_sum_w)
print("N_gal: %.0f"%mean_N_gal)

Sum-weighted: 108863.47
N_gal: 651506


### Create random files

In [58]:
N_ran_single = int(1.5*mean_N_gal)

In [61]:
full_ran = '/mnt/store1/oliverphilcox/PowerSpectra/qpm_randoms_50x.xyzwj'
with open(full_ran) as ran_in:
    for l,line in enumerate(ran_in):
        pass
N_ran = l+1

ran_x,ran_y,ran_z,ran_w = [np.zeros(N_ran) for _ in range(4)]
with open(full_ran) as ran_in:
    for l,line in enumerate(ran_in):
        if l%N_ran_single==0:
            print("%d percent complete"%(l/N_ran*100))
        split = line.split()
        ran_x[l] = float(split[0])
        ran_y[l] = float(split[1])
        ran_z[l] = float(split[2])
        ran_w[l] = float(split[3])

0 percent complete
3 percent complete
6 percent complete
9 percent complete
12 percent complete
15 percent complete
18 percent complete
21 percent complete
24 percent complete
27 percent complete
30 percent complete
33 percent complete
36 percent complete
39 percent complete
42 percent complete
45 percent complete
48 percent complete
51 percent complete
54 percent complete
57 percent complete
60 percent complete
63 percent complete
66 percent complete
69 percent complete
72 percent complete
75 percent complete
78 percent complete
81 percent complete
84 percent complete
87 percent complete
90 percent complete
93 percent complete
96 percent complete
99 percent complete


In [73]:
sum_w = np.sum(ran_w)

In [69]:
all_ind = np.arange(0,N_ran,1)
shuffle(all_ind)

In [80]:
for i in range(32):
    
    print("Creating random file %d"%(i+1))
    
    these_ind = all_ind[N_ran_single*i:N_ran_single*(i+1)]
    
    this_x = ran_x[these_ind]
    this_y = ran_y[these_ind]
    this_z = ran_z[these_ind]
    this_w = ran_w[these_ind]
    ## ensure weights have correct sum and sign
    this_w*=mean_sum_w/np.sum(this_w)*-1.
    
    outfile = '/mnt/store1/oliverphilcox/3PCF_QPM_v3/ran_%d'%i
    
    with open(outfile,"w") as out:
        for j in range(len(this_x)):
            out.write("%.8f %.8f %.8f %.8f\n"%(this_x[j],this_y[j],this_z[j],this_w[j]))

Creating random file 1
Creating random file 2
Creating random file 3
Creating random file 4
Creating random file 5
Creating random file 6
Creating random file 7
Creating random file 8
Creating random file 9
Creating random file 10
Creating random file 11
Creating random file 12
Creating random file 13
Creating random file 14
Creating random file 15
Creating random file 16
Creating random file 17
Creating random file 18
Creating random file 19
Creating random file 20
Creating random file 21
Creating random file 22
Creating random file 23
Creating random file 24
Creating random file 25
Creating random file 26
Creating random file 27
Creating random file 28
Creating random file 29
Creating random file 30
Creating random file 31
Creating random file 32


### Define mean nw

In [93]:
## First define volume

import scipy.spatial as ss
hull = ss.ConvexHull(np.vstack([ran_x,ran_y,ran_z]).T)
print('volume inside points is: ',hull.volume)
V=hull.volume # in (Mpc/h)^3

volume inside points is:  4098297811.0807405


In [95]:
this_fac=[]
for i in range(len(all_weights)):
    w = all_weights[i]
    n = (1./w-1.)/20000
    this_fac.append(np.mean(w**3*n**3)*V)

In [96]:
np.mean(this_fac),np.std(this_fac)

(0.00030429669778437283, 3.2028705997329453e-07)

In [98]:
np.save('/mnt/store1/oliverphilcox/3PCF_QPM_v3/nw3V',np.mean(this_fac))